# Create Fake Product Data

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from sqlalchemy import create_engine
#import sql_queries as sql
import configparser
import sys 
import os

In [2]:
script_dir = os.path.abspath('')
mymodule_dir = os.path.join( script_dir, '..', '..', 'database', 'table_setup' )
sys.path.append( mymodule_dir )

import sql_queries

In [3]:
# =============================================================================
# ### SQL CONNECTION
# =============================================================================
encoding = 'utf-16'

config_file_path = os.path.join( script_dir, '..', '..', 'database', 'connection' )
config_file_path += '/config.ini'
config = configparser.ConfigParser()
config.read(config_file_path, encoding = encoding)

HOST=config.get('jj_furniture','host')
USER=config.get('jj_furniture','user')   
PASSWORD=config.get('jj_furniture','password')
DATABASE=config.get('jj_furniture','database')

try:
    db_conn_string = 'postgres://{}:{}@{}/{}'.format(USER,PASSWORD,HOST,DATABASE)
    db_engine = create_engine(db_conn_string)

except Exception as e:
    print(e)
    
   

In [4]:
def get_ids(table_name):
    query = '''
    SELECT {}_id
    FROM {}
    '''

    results = db_engine.execute(query.format(table_name[:-1], table_name)).fetchall()
    return [id for id, in results]

In [5]:
material_ids = get_ids('materials')
color_ids = get_ids('colors')
description_ids = get_ids('descriptions')

In [6]:
products = [
    {
     'material_id': np.random.choice(material_ids),
     'color_id': np.random.choice(color_ids),
     'description_id': np.random.choice(description_ids),
     'pieces': np.random.randint(low = 1, high = 8, size = 1)[0],
     'cost': np.round(np.random.lognormal(mean = np.log(100), sigma = 1, size = 1)[0],2) # make sure to give same cost to manufacturer 
    }
    for x in range(1000)]


In [7]:
df_products = pd.DataFrame(products)

In [8]:
df_products.head() 

,color_id,cost,description_id,material_id,pieces
0,8,818.24,2,5,2
1,4,46.11,1,3,4
2,8,30.75,13,1,5
3,8,60.57,7,1,6
4,4,78.19,3,3,4


In [9]:
print(sql_queries.insert_product_table)


INSERT INTO products (material_id, color_id, description_id, pieces, cost)
VALUES (%s, %s, %s, %s, %s)



In [10]:
### insert customer records
for i, row in df_products.iterrows():
    if i % 100 == 0:
        print(i)
    db_engine.execute(sql_queries.insert_product_table, 
                        (row['material_id'],
                        row['color_id'],
                        row['description_id'],
                        row['pieces'],
                        row['cost']))

0
100
200
300
400
500
600
700
800
900
